In [1]:
import torch
import pickle
from argparse import Namespace
from torch.utils.data import DataLoader
from heterogt.utils.tokenizer import EHRTokenizer
from heterogt.utils.dataset import PreTrainEHRDataset, batcher, expand_level3
from heterogt.utils.seed import set_random_seed
from heterogt.model.model import HeteroGTPreTrain
from tqdm import tqdm

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.13.1
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
set_random_seed(123)

[INFO] Random seed set to 123


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
config = Namespace(
    dataset = "MIMIC-III",
    tasks = ["death", "readmission", "stay", "next_diag_6m", "next_diag_12m"], 
    task_index = 2,  # index of the task to train
    token_type = ["diag", "med", "lab", "pro"],
    special_tokens = ["[PAD]", "[CLS]"],
    attn_mask_dicts = [{1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 6:[2,3,4,5,6,7], 7:[2,3,4,5,6,7]}, 
                       {1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 6:[2,3,4,5,6,7], 7:[2,3,4,5,6,7]}],
    d_model = 64,
    num_heads = 4,
    batch_size = 32,
    lr = 1e-3,
    epochs = 25,
    early_stop_patience = 5,
    group_code_thre = 5,  # if there are group_code_thre diag codes belongs to the same group ICD code, then the group code is generated
    pretrain_mask_rate = 0.7,
    cls_ontology_weight = 5e-2,
    visit_ontology_weight = 5e-2,
    adm_type_weight = 5e-2,
)

In [5]:
full_data_path = f"./data_process/{config.dataset}-processed/mimic.pkl"  # for tokenizer
pretrain_data_path = f"./data_process/{config.dataset}-processed/mimic_pretrain.pkl" # for pretraining

In [6]:
ehr_full_data = pickle.load(open(full_data_path, 'rb'))
group_code_sentences = [expand_level3()[1]]
diag_sentences = ehr_full_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_full_data["NDC"].values.tolist()
lab_sentences = ehr_full_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_full_data["PRO_CODE"].values.tolist()
age_sentences = [[str(c)] for c in set(ehr_full_data["AGE"].values.tolist())] # important of [[]]
adm_type_sentences = ehr_full_data["ADMISSION_TYPE"].values.tolist()
max_admissions = ehr_full_data.groupby("SUBJECT_ID")["HADM_ID"].nunique().max()
config.max_num_adms = max_admissions
print(f"Max admissions per patient: {config.max_num_adms}")

Max admissions per patient: 8


In [7]:
tokenizer = EHRTokenizer(age_sentences, group_code_sentences, diag_sentences, med_sentences, lab_sentences, 
                         pro_sentences, special_tokens=config.special_tokens, adm_types_sentences=adm_type_sentences)
config.label_vocab_size = {"diag":tokenizer.token_number("diag"),     
                            "med":tokenizer.token_number("med"), 
                            "lab":tokenizer.token_number("lab"), 
                            "pro":tokenizer.token_number("pro")}
config.global_vocab_size = len(tokenizer.vocab.id2word)
config.age_vocab_size = tokenizer.token_number("age")
config.group_code_vocab_size = tokenizer.token_number("group")
print(f"Age vocabulary size: {config.age_vocab_size}")
print(f"Group code vocabulary size: {config.group_code_vocab_size}")

Age vocabulary size: 18
Group code vocabulary size: 19


In [8]:
# load pretrain data
ehr_pretrain_data = pickle.load(open(pretrain_data_path, 'rb'))
# load occurence data

In [9]:
pretrain_dataset = PreTrainEHRDataset(ehr_pretrain_data=ehr_pretrain_data, tokenizer=tokenizer, token_type=config.token_type,
                                      mask_rate=config.pretrain_mask_rate, group_code_thre=config.group_code_thre, max_num_adms=config.max_num_adms)
print("Number of pretrain samples:", len(pretrain_dataset))
pretrain_dataloader = DataLoader(pretrain_dataset, batch_size=config.batch_size, 
                                 collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain=True), shuffle=True)

Number of pretrain samples: 23146


In [10]:
model = HeteroGTPreTrain(tokenizer=tokenizer, token_types=config.token_type, d_model=config.d_model, num_heads=config.num_heads, layer_types=['gnn', 'tf', 'gnn', 'tf'], max_num_adms=config.max_num_adms, 
                     device=device, label_vocab_size=config.label_vocab_size, attn_mask_dicts=config.attn_mask_dicts,
                     use_cls_cat=True).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)

In [11]:
# 统一权重表：未指定的一律 1.0
loss_weights = {
    **{t: 1.0 for t in config.token_type},
    "cls_ontology": float(config.cls_ontology_weight),
    "visit_ontology": float(config.visit_ontology_weight),
    "adm_type": float(config.adm_type_weight)
}
loss_types = list(loss_weights.keys())
print(loss_weights)

{'diag': 1.0, 'med': 1.0, 'lab': 1.0, 'pro': 1.0, 'cls_ontology': 0.05, 'visit_ontology': 0.05, 'adm_type': 0.05}


In [12]:
for epoch in range(1, 1 + config.epochs):
    model.train()
    avg_total_loss = 0.0
    avg_loss_dict = {t: 0.0 for t in loss_types}         # 未加权分项
    avg_contrib_dict = {t: 0.0 for t in loss_types}      # 加权后贡献
    step_iter = tqdm(pretrain_dataloader, desc=f"Epoch {epoch:03d}", unit="batch")

    for step, batch in enumerate(step_iter):
        batch = [x.to(device) if isinstance(x, torch.Tensor) else x for x in batch]
        loss_dict = model(*batch)   # {loss_type: tensor}

        # 总 loss（加权）
        total_loss = sum(loss_weights[t] * loss_dict[t] for t in loss_types)

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # 累积
        avg_total_loss += total_loss.item()
        for t in loss_types:
            avg_loss_dict[t] += loss_dict[t].item()                       # 原始 loss
            avg_contrib_dict[t] += (loss_weights[t] * loss_dict[t]).item()  # 加权后贡献

    steps = step + 1
    avg_total_loss /= steps
    avg_loss_dict = {t: v / steps for t, v in avg_loss_dict.items()}
    avg_contrib_dict = {t: v / steps for t, v in avg_contrib_dict.items()}

    # 日志字符串
    raw_loss_str = ", ".join(f"{t}: {avg_loss_dict[t]:.4f}" for t in loss_types)
    contrib_str  = ", ".join(f"{t}: {avg_contrib_dict[t]:.4f}" for t in loss_types)

    print(f"[Epoch {epoch:03d}] Weighted total loss: {avg_total_loss:.4f}\n"
          f"  Raw losses      | {raw_loss_str}\n"
          f"  Contributions   | {contrib_str}")

Epoch 001:   0%|          | 0/724 [00:00<?, ?batch/s]

Epoch 001: 100%|██████████| 724/724 [00:51<00:00, 14.13batch/s]


[Epoch 001] Weighted total loss: 0.2961
  Raw losses      | diag: 0.0264, med: 0.1056, lab: 0.0834, pro: 0.0253, cls_ontology: 0.4266, visit_ontology: 0.4245, adm_type: 0.2573
  Contributions   | diag: 0.0264, med: 0.1056, lab: 0.0834, pro: 0.0253, cls_ontology: 0.0213, visit_ontology: 0.0212, adm_type: 0.0129


Epoch 002: 100%|██████████| 724/724 [00:46<00:00, 15.44batch/s]


[Epoch 002] Weighted total loss: 0.2463
  Raw losses      | diag: 0.0175, med: 0.0921, lab: 0.0726, pro: 0.0160, cls_ontology: 0.3856, visit_ontology: 0.3866, adm_type: 0.1909
  Contributions   | diag: 0.0175, med: 0.0921, lab: 0.0726, pro: 0.0160, cls_ontology: 0.0193, visit_ontology: 0.0193, adm_type: 0.0095


...

[Epoch 023] Weighted total loss: 0.2112
  Raw losses      | diag: 0.0151, med: 0.0757, lab: 0.0673, pro: 0.0120, cls_ontology: 0.3467, visit_ontology: 0.3485, adm_type: 0.1287
  Contributions   | diag: 0.0151, med: 0.0757, lab: 0.0673, pro: 0.0120, cls_ontology: 0.0173, visit_ontology: 0.0174, adm_type: 0.0064


Epoch 024: 100%|██████████| 724/724 [00:46<00:00, 15.61batch/s]


[Epoch 024] Weighted total loss: 0.2106
  Raw losses      | diag: 0.0150, med: 0.0754, lab: 0.0672, pro: 0.0119, cls_ontology: 0.3459, visit_ontology: 0.3480, adm_type: 0.1272
  Contributions   | diag: 0.0150, med: 0.0754, lab: 0.0672, pro: 0.0119, cls_ontology: 0.0173, visit_ontology: 0.0174, adm_type: 0.0064


Epoch 025: 100%|██████████| 724/724 [00:50<00:00, 14.37batch/s]

[Epoch 025] Weighted total loss: 0.2104
  Raw losses      | diag: 0.0150, med: 0.0752, lab: 0.0671, pro: 0.0119, cls_ontology: 0.3463, visit_ontology: 0.3484, adm_type: 0.1290
  Contributions   | diag: 0.0150, med: 0.0752, lab: 0.0671, pro: 0.0119, cls_ontology: 0.0173, visit_ontology: 0.0174, adm_type: 0.0065


In [13]:
import os
exp_name = (
    f"{config.dataset}-{config.pretrain_mask_rate}-{config.d_model}-{config.cls_ontology_weight}-{config.visit_ontology_weight}-{config.adm_type_weight}"
)
save_path = "./pretrained_models/" + exp_name
if not os.path.exists(save_path):
    os.makedirs(save_path)
torch.save(model.cpu().state_dict(), f"{save_path}/pretrained_model.pt")
print("Save model:", exp_name)

Save model: MIMIC-III-0.7-64-0.05-0.05-0.05
